In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
!pip install kagglehub

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivamb/netflix-shows")

print("Path to dataset files:", path)


C:\Users\Sonu Pal\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████████████████████████████████████████████████████████████████████████| 1.34M/1.34M [00:01<00:00, 807kB/s]

Extracting files...
Path to dataset files: C:\Users\Sonu Pal\.cache\kagglehub\datasets\shivamb\netflix-shows\versions\5


In [9]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Sonu Pal\.cache\kagglehub\datasets\shivamb\netflix-shows\versions\5\netflix_titles.csv")
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [10]:
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [11]:
print("\nMissing values count per column:")
print(df.isnull().sum())
print("\nMissing values percentage per column:")
print((df.isnull().sum() / len(df)) * 100)


Missing values count per column:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

Missing values percentage per column:
show_id          0.000000
type             0.000000
title            0.000000
director        29.908028
cast             9.367549
country          9.435676
date_added       0.113546
release_year     0.000000
rating           0.045418
duration         0.034064
listed_in        0.000000
description      0.000000
dtype: float64


In [13]:
df['director'] = df['director'].fillna('Unknown')
df['cast'] = df['cast'].fillna('Unknown')

In [14]:
# 1. Calculate the mode (most frequent country)
mode_country = df['country'].mode()[0]

# 2. Fill missing values with the mode
df['country'] = df['country'].fillna(mode_country)

print("Missing values in 'country' after mode imputation:", df['country'].isnull().sum())

Missing values in 'country' after mode imputation: 0


In [15]:
# Drop rows where 'date_added' is null
df.dropna(subset=['date_added'], inplace=True)

print("Remaining rows after dropping date_added nulls:", df.shape[0])

Remaining rows after dropping date_added nulls: 8797


In [16]:
# Drop rows where 'rating' or 'duration' are null.
# This cleans up the very few remaining nulls (4 in rating, 3 in duration from the start).
df.dropna(subset=['rating', 'duration'], inplace=True)

# Final check of the dataset size and missing values
print(f"Remaining rows after dropping rating/duration nulls: {df.shape[0]}")
print("\nFinal Missing values check (All should be 0):")
print(df.isnull().sum())

Remaining rows after dropping rating/duration nulls: 8790

Final Missing values check (All should be 0):
show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


In [17]:
# Convert all column names to snake_case
df.columns = df.columns.str.lower().str.replace(' ', '_')

print("\nNew Column Names:", df.columns.tolist())


New Column Names: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [19]:
df['date_added'] = df['date_added'].str.strip()
df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y')
print(f"Data type of date_added after conversion: {df['date_added'].dtype}")

Data type of date_added after conversion: datetime64[ns]


In [20]:
# Strip any leading/trailing spaces and apply Title Case for uniformity in country names
df['country'] = df['country'].str.strip().str.title()

In [21]:
print("\nFinal Data Info:")
df.info()


Final Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 8790 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8790 non-null   object        
 1   type          8790 non-null   object        
 2   title         8790 non-null   object        
 3   director      8790 non-null   object        
 4   cast          8790 non-null   object        
 5   country       8790 non-null   object        
 6   date_added    8790 non-null   datetime64[ns]
 7   release_year  8790 non-null   int64         
 8   rating        8790 non-null   object        
 9   duration      8790 non-null   object        
 10  listed_in     8790 non-null   object        
 11  description   8790 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 892.7+ KB


In [22]:
# Strip any leading/trailing spaces and apply Title Case for uniformity in country names
df['country'] = df['country'].str.strip().str.title()

In [23]:
df.to_csv('netflix_titles_cleaned.csv', index=False)
print("\nCleaned dataset saved as 'netflix_titles_cleaned.csv'")


Cleaned dataset saved as 'netflix_titles_cleaned.csv'
